# Titanic - Machine Learning from Disaster

<img src="https://storage.googleapis.com/kaggle-competitions/kaggle/3136/logos/header.png" width=900>

- Vamos utilizar os [dados disponíveis no Kaggle](https://www.kaggle.com/competitions/titanic)
    - É um dataset de **competição**
    - O resultado é avaliado através da **acurácia**:
        - _"Sua pontuação é a porcentagem de passageiros que você prevê corretamente. Isso é conhecido como acurácia."_

### Importando novamente as bases e fazendo o tratamento dos dados
- Vamos apenas replicar o que fizemos no **[primeiro arquivo dessa análise](https://github.com/lucaslealx/Titanic/blob/main/Parte1.ipynb)** (para visualizar a parte 1 completa, visite esse link)

In [333]:
# Importando o pandas
import pandas as pd

In [334]:
# Visualizando a base de treino
treino = pd.read_csv('train.csv')
treino.head(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S


In [335]:
# Visualizando a base de teste
teste = pd.read_csv('test.csv')
teste.head(3)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q


- **Fazendo o mesmo tratamento inicial que fizemos na aula anterior**

In [336]:
# Eliminando as colunas com elevada cardinalidade
treino = treino.drop(['Name','Ticket','Cabin'],axis=1)
teste = teste.drop(['Name','Ticket','Cabin'],axis=1)

In [337]:
# Usando a média para substituir valores nulos na coluna de idade
treino.loc[treino.Age.isnull(),'Age'] = treino.Age.mean()
teste.loc[teste.Age.isnull(),'Age'] = teste.Age.mean()

In [338]:
# Tratando a coluna Embarked da base de treino usando a moda 
treino.loc[treino.Embarked.isnull(),'Embarked'] = treino.Embarked.mode()[0]

In [339]:
# E também a coluna Fare da base de teste usando a média
teste.loc[teste.Fare.isnull(),'Fare'] = teste.Fare.mean()

### Podemos agora entender as colunas de texto
- Vamos agora **adicionar um novo tratamento das colunas de texto**

In [340]:
# Verificando as colunas de texto na base de treino
treino.columns[treino.dtypes == 'object']

Index(['Sex', 'Embarked'], dtype='object')

In [341]:
# Verificando os valores na coluna Sex
treino.Sex.value_counts()

Sex
male      577
female    314
Name: count, dtype: int64

In [342]:
# e na coluna Embarked
treino.Embarked.value_counts()

Embarked
S    646
C    168
Q     77
Name: count, dtype: int64

- Para tratar a coluna Sex, podemos criar uma nova coluna chamada **"Male_Check"** que **vai receber 1 se o gênero for masculino e 0 se o gênero for feminino**

In [343]:
# Usando uma lambda function para fazer esse tratamento
treino['Male_check'] = treino.Sex.apply(lambda x: 1 if x == 'male' else 0) 

In [344]:
# Verificando os valores
treino[['Sex','Male_check']].value_counts()

Sex     Male_check
male    1             577
female  0             314
Name: count, dtype: int64

- O próximo passo é fazer o mesmo para a base de teste

In [345]:
# Usando uma lambda function para fazer esse tratamento
teste['Male_check'] = teste.Sex.apply(lambda x: 1 if x == 'male' else 0) 

In [346]:
# Verificando os valores
teste[['Sex','Male_check']].value_counts()

Sex     Male_check
male    1             266
female  0             152
Name: count, dtype: int64

- Agora, para tratar a coluna **Embarked** usando o **[OneHotEncoder](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html)** que irá **criar uma nova coluna para cada um dos rótulos da coluna original**

In [347]:
# Importando o OneHotEncoder
from sklearn.preprocessing import OneHotEncoder

In [348]:
# Criando o encoder
enc = OneHotEncoder(handle_unknown='ignore', dtype='int32')

In [349]:
# Fazendo o fit com os dados
enc.fit(treino[['Embarked']])

OneHotEncoder(dtype='int32', handle_unknown='ignore')

In [350]:
# Fazendo a transformação
enc.transform(treino[['Embarked']]).toarray()

array([[0, 0, 1],
       [1, 0, 0],
       [0, 0, 1],
       ...,
       [0, 0, 1],
       [1, 0, 0],
       [0, 1, 0]])

In [351]:
# Transformando esse resultado em um DataFrame
enc_df = pd.DataFrame(data= enc.transform(treino[['Embarked']]).toarray(), columns=enc.get_feature_names_out())

In [352]:
# Podemos agora adicionar essa coluna na nossa base de teste
treino = pd.concat([treino, enc_df], axis=1)

In [353]:
# Verificando os valores
treino[['Embarked','Embarked_C','Embarked_Q','Embarked_S']].value_counts()

Embarked  Embarked_C  Embarked_Q  Embarked_S
S         0           0           1             646
C         1           0           0             168
Q         0           1           0              77
Name: count, dtype: int64

- Agora vamos **fazer o mesmo para a base de teste usando o encoder ohe que criamos acima**

In [354]:
# Transformando esse resultado em um DataFrame
enc_test = pd.DataFrame(data= enc.transform(teste[['Embarked']]).toarray(), columns=enc.get_feature_names_out())

In [355]:
# Adicionando o resultado na base de teste
teste = pd.concat([teste, enc_test], axis=1)

In [356]:
# Verificando também os valores
teste[['Embarked','Embarked_C','Embarked_Q','Embarked_S']].value_counts()

Embarked  Embarked_C  Embarked_Q  Embarked_S
S         0           0           1             270
C         1           0           0             102
Q         0           1           0              46
Name: count, dtype: int64

### Usando essa nova base no modelo

In [357]:
# Visualizando a base
treino.head(3)

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Male_check,Embarked_C,Embarked_Q,Embarked_S
0,1,0,3,male,22.0,1,0,7.2500,S,1,0,0,1
1,2,1,1,female,38.0,1,0,71.2833,C,0,1,0,0
2,3,1,3,female,26.0,0,0,7.9250,S,0,0,0,1


In [358]:
# Podemos então apagar essas duas colunas que já tratamos
treino = treino.drop(['Sex', 'Embarked'], axis=1)
teste = teste.drop(['Sex', 'Embarked'], axis=1)

- Podemos selecionar os mesmos modelos que vimos anteriormente (consulte arquivo da **[parte 1](https://github.com/lucaslealx/Titanic/blob/main/Parte1.ipynb)**)
    - **Árvore de classificação**
        - https://scikit-learn.org/stable/modules/tree.html#classification
    - **Classificação dos vizinhos mais próximos**
        - https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html#sklearn.neighbors.KNeighborsClassifier
    - **Regressão Logística**
        - https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html#sklearn.linear_model.LogisticRegression
- Antes de usar os algoritmos, precisamos separar a base de treino em **treino e validação**
    - Vamos fazer isso utilizando o **train_test_split**
        - https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

In [359]:
# Importando o train_test_split
from sklearn.model_selection import train_test_split

In [360]:
# Separando a base de treino em X e y
X = treino.drop(['PassengerId','Survived'],axis=1)
y = treino.Survived

In [361]:
# Separando em treino e validação
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.33, random_state=42)

- Para a **árvore de classificação**

In [362]:
# Fazendo a importação
from sklearn import tree

In [363]:
# Criando o classificador
clf_ac = tree.DecisionTreeClassifier(random_state=42)

In [364]:
# Fazendo o fit com os dados
clf_ac = clf_ac.fit(X_train,y_train)

In [365]:
# Fazendo a previsão
y_pred_ac = clf_ac.predict(X_val)

- Para o **KNeighborsClassifier**

In [366]:
# Importando
from sklearn.neighbors import KNeighborsClassifier

In [367]:
# Criando o classificador
clf_knn = KNeighborsClassifier(n_neighbors=3)

In [368]:
# Fazendo o fit com os dados
clf_knn = clf_knn.fit(X_train,y_train)

In [369]:
# Fazendo a previsão
y_pred_knn = clf_knn.predict(X_val)

- E para a **Regressão Logística**

In [370]:
# Importando
from sklearn.linear_model import LogisticRegression

In [371]:
# Criando o classificador
clf_rl = LogisticRegression(random_state=42,max_iter=1000)

In [372]:
# Fazendo o fit com os dados
clf_rl = clf_rl.fit(X_train,y_train)

In [373]:
# Fazendo a previsão
y_pred_rl = clf_rl.predict(X_val)

- Vamos novamente **avaliar os modelos**
    - Acurácia (método de avaliação usado na competição):
        - https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html
    - Matriz de confusão (ajuda a visualizar a distribuição dos erros):
        - https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html

- Avaliando a **acurácia**

In [374]:
# Importando
from sklearn.metrics import accuracy_score

In [375]:
# Para a árvore
accuracy_score(y_val, y_pred_ac)

0.7491525423728813

In [376]:
# Para o knn
accuracy_score(y_val, y_pred_knn)

0.7152542372881356

In [377]:
# Para a regressão logística
accuracy_score(y_val, y_pred_rl)

0.8169491525423729

- Avaliando a **matriz de confusão**

In [378]:
# Importando
from sklearn.metrics import confusion_matrix

In [379]:
# Para a árvore
confusion_matrix(y_val, y_pred_ac)

array([[138,  37],
       [ 37,  83]], dtype=int64)

In [380]:
# Para o knn
confusion_matrix(y_val, y_pred_knn)

array([[147,  28],
       [ 56,  64]], dtype=int64)

In [381]:
# Para a regressão logística
confusion_matrix(y_val, y_pred_rl)

array([[153,  22],
       [ 32,  88]], dtype=int64)

### Fazendo a previsão para os dados de teste
- Vamos usar o modelo com melhor precisão para fazer o predict na base de teste

In [382]:
# Visualizando o X_train
X_train.head(3)

,Pclass,Age,SibSp,Parch,Fare,Male_check,Embarked_C,Embarked_Q,Embarked_S
6,1,54.000000,0,0,51.8625,1,0,0,1
718,3,29.699118,0,0,15.5000,1,0,1,0
685,2,25.000000,1,2,41.5792,1,1,0,0


In [383]:
# Visualizando a base de teste
teste.head(3)

,PassengerId,Pclass,Age,SibSp,Parch,Fare,Male_check,Embarked_C,Embarked_Q,Embarked_S
0,892,3,34.5,0,0,7.8292,1,0,1,0
1,893,3,47.0,1,0,7.0000,0,0,0,1
2,894,2,62.0,0,0,9.6875,1,0,1,0


In [384]:
# Para a base de teste ser igual a base de treino, precisamos eliminar a coluna de id
X_teste = teste.drop('PassengerId', axis=1)

In [385]:
# Utilizando a regressão logística na base de teste
y_pred = clf_rl.predict(X_teste)

In [386]:
# Criando uma nova coluna com a previsão na base de teste
teste['Survived'] = y_pred

In [390]:
# Selecionando apenas a coluna de Id e Survived para fazer o envio
teste_envio = teste[['PassengerId','Survived']]

In [391]:
# Exportando para um csv
teste_envio.to_csv('resultados2.csv', index=False)